In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("C:/Users/lauta/Desktop/Internship DealRoom/data_scientist_duplicate_detection.csv")
df1 = df.copy()

#### My approach is to look for duplicates with the names of the company. But for this to work, the names of the companies have to be standarized as much as possible, as to minimize the differences coming from the data entry process.

In [3]:
spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]

df1.name = df1.name.str.strip()
df1.name = df1.name.str.lower()
df1.name = df1.name.str.replace(',', ' ')
for char in spec_chars:
    df1.name = df1.name.str.replace(char,'')

#### Now that we have "clean" names, It would be easy to look up the duplicates. Nevertheless, the assignment is also asking to pinpoint the row and the columns on which each duplicate is equal to the original observations. For this, I decided to create to different dataset with the same method but a different parameter. The first dataset (df_dup_f) will contain the second duplication, and the second dataset will contain the original observation.

In [4]:
df_dup_f = df1[df1.duplicated(['name'], keep='first')].reset_index()
df_dup_l = df1[df1.duplicated(['name'], keep='last')].reset_index()

In [5]:
#empty lists that will eventually fill the final dataset with the index of the row on which it is duplicated
#and the columns on which they are equal
dup_index = []
dup_columns = []

#for every name there is in the duplicate dataframe
for company in df_dup_f.name:
    #get the index of the row of the second list (that contains the first duplicate) and append it to the list of indexes
    dup_index.append(df_dup_l[df_dup_l['name'] == company].iloc[0]['index'])
    #get the names of the columns on which the row of the first duplicate dataframe is equal to the duplicate of
    #the second duplicate dataframe and append it to the list of duplicated columns
    
    dup_columns.append(list(df_dup_f.columns[np.where(df_dup_f[df_dup_f['name'] == company].iloc[0] == df_dup_l[df_dup_l['name'] == company].iloc[0])]))

In [6]:
#create the new columns for the information about duplicated indexes and columns
df_dup_f['duplicated with (index of row in original table)'] = dup_index
df_dup_f['duplicated on'] = dup_columns
df_dup_f.head()

,index,id,name,tagline,industry,industry_2,type,address,street,street_number,zip,country,city,sectors,duplicated with (index of row in original table),duplicated on
0,34,1315405,possip,Engagement platform giving schools a pulse on ...,Kids,NaN,2,"United States, Nashville",NaN,NaN,NaN,United States,Nashville,"Student,automated process,platform,SaaS,Automa...",29,"[name, industry, type, country, city]"
1,48,85919,spark networks,"Spark Networks – A Leading Provider of Iconic,...",Dating,NaN,4,"11150 CA-2 #600, Los Angeles, CA 90025, USA",California 2,11150,NaN,United States,West Los Angeles,"presentation,network,Intellectual Property,pre...",43,"[name, industry, address, street, street_numbe..."
2,65,1314213,the johns hopkins bloomberg school of public h...,"Research, education, and practice to find solu...",Kids,Education,8,"United States, Baltimore",NaN,NaN,NaN,United States,Baltimore,NaN,9,"[name, tagline, industry, address, country, city]"
3,82,1733948,budge studios,Developer of mobile applications to entertain ...,Kids,NaN,2,"Canada, Montreal",NaN,NaN,H2L,Canada,Montreal,app,7,"[name, industry, type, country, city]"
4,423,1233426,allunga advisory,Boutique investment bank with an established t...,NaN,NaN,3,"19-29 Martin Pl, Sydney NSW 2000, Australia",Martin Place,19-29,2000,Australia,Sydney,NaN,234,"[name, tagline, type, country, city]"


#### Now we have a dataframe with the duplicates (ignoring the first observation) with the index of the first observation from the original dataset to check it, and the names of the columns on which this duplicate is equal to the observation of the first observation in the original dataset.

In [7]:
df_dup_f.to_csv('C:/Users/lauta/Desktop/Internship DealRoom/Duplicates.csv', index=False)

#### However, this solution is incomplete. Some observations were duplicated on more than one time, meaning that the information about the index and columns on which is duplicated for some values will be wrong.

In [8]:
df_dup_f.name.value_counts()

latch                              2
caterwings                         2
koko                               2
oxygen                             2
grounded                           2
                                  ..
computerrx                         1
appreciation engine                1
western states investment group    1
geko navsat                        1
droplit                            1
Name: name, Length: 887, dtype: int64